In [18]:
import pandas as pd
import numpy as np
# fancy graph
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
# nlp
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.tokenizer import Tokenizer
import en_core_web_sm
# reproduce
np.random.seed(42)
SEED = 42


In [11]:
!ls

cleaned_data  graphs			project.ipynb  scraped_data
eda.ipynb     presentation_script.docx	README.md      testing.ipynb


In [12]:
import en_core_web_lg

In [13]:
nlp = en_core_web_lg.load()


In [19]:
tokenizer = Tokenizer(nlp.vocab)


In [20]:
tfidf = TfidfVectorizer(min_df=1,
                        max_df=0.95,
                        tokenizer=tokenizer)


In [67]:
df = pd.read_csv("cleaned_data/chanellos.csv")

In [54]:
df = df.dropna()

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test = train_test_split(df)
X_train.shape, X_test.shape

((981, 11), (328, 11))

In [68]:
df1 = pd.read_csv("cleaned_data/calz.csv")
df.shape

(2088, 11)

In [59]:
df1 = df1.dropna()
df.shape

(1309, 11)

In [27]:
print(df.columns)
df1.columns

Index(['Unnamed: 0', 'time', 'customer_name', 'review_content',
       'review_rating', 'store_name', 'store_rating', 'store_address',
       'tot_num_rate', 'latitude', 'longitude'],
      dtype='object')


Index(['Unnamed: 0', 'time', 'customer_name', 'review_content',
       'review_rating', 'store_name', 'store_rating', 'store_address',
       'num_to_rate', 'owner_response', 'owner_response_time', 'latitude',
       'longitude'],
      dtype='object')

In [28]:
df.head()

,Unnamed: 0,time,customer_name,review_content,review_rating,store_name,store_rating,store_address,tot_num_rate,latitude,longitude
0,0,1.581293e+09,lauren sams,Great and fast service with excellent food.,5,Chanello's Pizza #33,3.1,1122 George Washington Hwy N,124,36.782,-76.338
2,2,1.550945e+09,Janel Brosseit,This rating goes more towards the food. Just r...,2,Chanello's Pizza #33,3.1,1122 George Washington Hwy N,124,36.782,-76.338
4,4,1.456272e+09,Alexandra Kehl,"Waited over an hour for our food, then the man...",1,Chanello's Pizza #33,3.1,1122 George Washington Hwy N,124,36.782,-76.338
5,5,1.550945e+09,Big A,"No thanks owner, u only get so many chances, d...",1,Chanello's Pizza #33,3.1,1122 George Washington Hwy N,124,36.782,-76.338
6,6,1.582243e+09,Brandon Davis,"Friendly staff, always willing to help a custo...",5,Chanello's Pizza #33,3.1,1122 George Washington Hwy N,124,36.782,-76.338


In [29]:
df1.head()

,Unnamed: 0,time,customer_name,review_content,review_rating,store_name,store_rating,store_address,num_to_rate,owner_response,owner_response_time,latitude,longitude
0,0,1.551463e+09,Jeanette Zeeb,It's ok,3,Cal'z Pizza,4.0,3678 Sewells Point Rd,322,Response from the owner,a year ago,36.918,-76.2
5,5,1.551463e+09,Rob Craft,In town for a daughter visit and this was wher...,5,Cal'z Pizza,4.0,3678 Sewells Point Rd,322,Response from the owner,a year ago,36.918,-76.2
6,6,1.551463e+09,Emani Arps,GREAT COSTMER SERVICES,5,Cal'z Pizza,4.0,3678 Sewells Point Rd,322,Response from the owner,11 months ago,36.918,-76.2
7,7,1.551463e+09,Renee Cavanagh,Took over an hour to arrive (im not even 5-6 m...,2,Cal'z Pizza,4.0,3678 Sewells Point Rd,322,Response from the owner,a year ago,36.918,-76.2
8,8,1.573472e+09,7 PRS,Food is good and service is friendly. Not my f...,5,Cal'z Pizza,4.0,3678 Sewells Point Rd,322,Response from the owner,2 years ago,36.918,-76.2


In [33]:
df1.drop(['owner_response', 'owner_response_time'], axis=1, inplace=True)

In [35]:
assert len(df1.columns) == len(df.columns)

In [88]:
# looking for what makes a good review
df1['target'] = df1['review_rating'] >= 3
df['target'] = df['review_rating'] >= 3

# looking at the oposite thus the anti
df1['anti_target'] = df1['review_rating'] <= 3
df['anti_target'] = df['review_rating'] <= 3

# changing bools to ints
df['target'] = df['target'].map({True: 1, False: 0})
df1['target'] = df1['target'].map({True: 1, False: 0})
df['anti_target'] = df['anti_target'].map({True: 1, False: 0})
df1['anti_target'] = df1['anti_target'].map({True: 1, False: 0})


In [89]:
from sklearn.model_selection import train_test_split
X_train1, y_train1, X_test1, y_test1 = train_test_split(df1.drop(['target', 'anti_target', 'Unnamed: 0', 'customer_name'], axis=1),
                                                                 df1['target'])
X_train, y_train, X_test, y_test = train_test_split(df.drop(['target', 'anti_target', 'Unnamed: 0', 'customer_name'], axis=1),
                                                            df['target'])
print(f"Training shapes: df-0{X_train.shape}\tdf-1{X_train1.shape},\tTest shape: df-0{X_test.shape}\tdf-1{X_test1.shape}")

Training shapes: df-0(1566, 9)	df-1(780, 11),	Test shape: df-0(1566,)	df-1(780,)


In [85]:
print("training set full", f"X{X_train.shape}", f"X1{X_train1.shape}", f"y{y_train.shape}", f"y1{y_train1.shape}", sep='\n')

training set full
X(1566, 9)
X1(780, 11)
y(522, 9)
y1(261, 11)


In [90]:
y_train = y_train['target']

KeyError: 'target'

In [29]:

tfidf.fit(X_train.review_content)


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.95, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<spacy.tokenizer.Tokenizer object at 0x7f17452594d0>,
                use_idf=True, vocabulary=None)

In [31]:
# make vector encodings of the data from the chanellos dataset
X_train_vect = tfidf.transform(X_train)
X_test_vect = tfidf.transform(X_test)


In [34]:
# make sparse matrixs from the vectorizations
dtm_train = X_train_vect.todense()
dtm_test = X_test_vect.todense()


In [46]:
from sklearn.neighbors import NearestNeighbors


In [47]:
nn = NearestNeighbors(n_jobs=-1)
nn.fit(dtm_train)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=-1, n_neighbors=5, p=2, radius=1.0)

(array([[0., 0., 0., 0., 0.]]), array([[8, 9, 6, 5, 7]]))

"This was my first time ordering from here and I must say it was quite good. The all meat pizza was good for a local chain. I'd prefer them over the larger chains."